In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sympy import sieve
import time

from common import *

In [2]:
p = problem(base_path='./solution/base2.csv')

In [3]:
p.solve()

Start iteration 1
a: 1 | saving: -33.777 | cumsave: -33.777
a: 1 | saving: -11.610 | cumsave: -45.386
a: 4796 | saving: -0.176 | cumsave: -45.562
a: 4934 | saving: -0.970 | cumsave: -46.532
a: 15033 | saving: -0.590 | cumsave: -47.122
a: 15034 | saving: -3.560 | cumsave: -50.682
a: 20698 | saving: -0.215 | cumsave: -50.896
a: 22746 | saving: -0.804 | cumsave: -51.701
a: 26458 | saving: -1.616 | cumsave: -53.317
a: 26511 | saving: -1.037 | cumsave: -54.354
a: 30362 | saving: -6.685 | cumsave: -61.038
a: 31446 | saving: -0.555 | cumsave: -61.593
a: 31484 | saving: -1.392 | cumsave: -62.985
a: 31863 | saving: -0.520 | cumsave: -63.506
a: 32691 | saving: -0.709 | cumsave: -64.214
a: 32811 | saving: -0.955 | cumsave: -65.169
a: 33438 | saving: -0.104 | cumsave: -65.273
a: 34322 | saving: -4.788 | cumsave: -70.062
a: 34322 | saving: -2.987 | cumsave: -73.049
a: 35492 | saving: -1.485 | cumsave: -74.534
a: 38400 | saving: -1.618 | cumsave: -76.152
a: 38406 | saving: -0.725 | cumsave: -76.877


In [5]:
p.score(p.base)

1516780.6926535096

In [11]:
3600*24*3

259200

In [9]:
p.score(cls.solution)

1516537.4662957683

# UTILITY

In [10]:
#write result
pd.DataFrame({'Path':cls.solution}).to_csv('./solution/submit_base2.csv',index=False)

In [ ]:
#read LKH result into solution
f = open('./solver/tsp_solution.csv','r')
line = 1
content = '123'
sol = []
while content != '':
    content = f.readline()
    if line >=7:
        
        sol.append(content)
    line+=1
f.close()

sol = sol[:-2]
sol = [int(line.split('\n')[0]) for line in sol]
sol[-1] = 1
solution = np.array(sol)
solution-=1

In [ ]:
#make .tsp problem
fname = path + 'problem/santa.tsp'
with open(fname, 'w') as f:
    f.write('NAME : santa\n')
    f.write('COMMENT : santa\n')
    f.write('TYPE : TSP\n')
    f.write('DIMENSION : %d\n' % n)
    f.write('EDGE_WEIGHT_TYPE : EUC_2D\n')
    f.write('NODE_COORD_SECTION\n')
    for idx,node in enumerate(nodes):
        x,y=node*100000
        f.write('%s %s %s\n' % (idx+1,x,y))
    f.write('EOF\n')

In [ ]:
#read neos html
fname = path + 'solution/neos2.html'
l = 1
sol = []
with open(fname, 'r') as f:
    while l< 207274:
        line = f.readline()
        if l>=9505:
            sol.append(int(line.split(' ')[0]))
        l += 1
sol.append(0)
solution = np.array(sol)

In [ ]:
#2-opt manual check calcucation
a,d = 185337,186129 
a_pre,d_nex = a-1,d+1

node_a,node_d,node_a_pre,node_d_nex = solution[[a,d,a_pre,d_nex]]
old = score(solution)[a_pre:d_nex].sum()

to_d  = distance(nodes[node_a_pre],nodes[node_d])
from_a = distance(nodes[node_a],nodes[node_d_nex])
check = (d+1)%10 ==0 and node_a not in prime
print(check)
if check:
    from_a*=1.1

partial = solution[185337:186129+1][::-1]
total = score(partial,offset = a%10).sum()

In [ ]:
#2-opt functions
def calc_reverse(left, nodes, solution):
    r_solution = solution[left:-1][::-1]
    reverse_dist = []
    for offset in range(10):
        reverse_dist.append(np.cumsum(score(nodes,r_solution,offset)[::-1])[::-1])
    reverse_dist = np.array(reverse_dist)
    
    return reverse_dist

def calc_old_dist(nodes, solution, from_node):
    offset = from_node%10
    return np.cumsum(score(nodes,solution[from_node:],offset=offset))

def calc_middle(reverse_dist, left):
    middle = np.zeros(shape=(reverse_dist.shape[1]))
    offset = left%10
    
    for o in range(10):
        order = (offset-o)%10
        using_reverse = reverse_dist[order]
        middle[o::10] = using_reverse[o::10]
    
    reverse_dist[:,:-1] -= reverse_dist[:,-1:]
    reverse_dist = reverse_dist[:,:-1]
    return middle[::-1], reverse_dist

def calc_connector(nodes, solution_nodes, roller, left, node_before_a, node_a, prime_set):
    to_d = roller[:-1].copy()
    offset = left%10
    if node_before_a not in prime_set and offset == 0:
        to_d *= 1.1
        
    roller = distance(nodes[node_a],solution_nodes[left+2:])
    if node_a in prime_set:
        return to_d, roller, roller  
    else:
        start_leg = (10-left+2)%10
        from_a = roller.copy()
        from_a[start_leg::10] *= 1.1
        return to_d, from_a, roller
    
def calc_save(from_a,to_d,middle,old_dist):
    save = from_a+to_d-old_dist[2:]+middle
    old_dist[1:] -= old_dist[0]
    old_dist = old_dist[1:]
    
    place = np.argmin(save)
    saving = save[place]
    return place, saving, old_dist

def calc_basis(start_at, nodes, solution, solution_nodes):
    reverse_dist = calc_reverse(start_at, nodes, solution)
    old_dist = calc_old_dist(nodes, solution,from_node=start_at-1)
    roller = distance(nodes[solution[start_at-1]],solution_nodes[start_at-1+2:])
    
    return reverse_dist, old_dist, roller

def excute_swap(left,place,solution,solution_nodes):
    target = place+1+left
    
    solution[left:target+1] = solution[target:left-1:-1]
    solution_nodes[left:target+1] = solution_nodes[target:left-1:-1]
    

In [ ]:
bests = []
dids = []

asd = nodes[solution]
df = pd.DataFrame(asd)
df.to_csv('asd.csv')
solution_nodes = pd.read_csv('asd.csv').iloc[:,1:].values

start = time.time()

i=1
iter_num = 5
start_at = 1
run_through = 2000

fast = 

while i <= iter_num:
    iter_start = time.time()
    best = []
    did = 0
    
    reverse_dist, old_dist, roller = calc_basis(start_at, nodes, solution, solution_nodes)

    for left in range(start_at, run_through):
        node_before_a, node_a = solution[left-1:left+1]

        middle, reverse_dist = calc_middle(reverse_dist, left)

        to_d, from_a, roller =  calc_connector(nodes, solution_nodes, roller, left, node_before_a, node_a, prime_set)

        place, saving, old_dist = calc_save(from_a,to_d,middle,old_dist)
        
        if saving<0:
            
            best.append(saving)
            did += saving
            target = place+1+left
            
            #execute swap
            excute_swap(left,place,solution,solution_nodes)
            print('a: %s | saving: %.3f | cumsave: %.3f'%(left, saving, did))
            
            #reset infos
            reverse_dist, old_dist, roller = calc_basis(left+1, nodes, solution, solution_nodes)

    #pd.DataFrame({'Path':solution}).to_csv('./solution/neos_iter_%s.csv'%(i+1),index=False)
    bests.append(best)
    dids.append(did)
    print('iter %s done, total swap %s, saves %.3f, time %.1f'%(i,len(best),did,time.time()-iter_start))
    print(score(nodes,solution).sum())
    print('####################################')
    i+=1
    if len(best) == 0:
        print('zero')
        break
print(time.time()-start)

# node swap

In [4]:

nodes = p.nodes
solution = p.solution
prime = p.prime
n = len(nodes)

In [5]:
start = time.time()

#every iter
coor = nodes[solution]
dist = score(nodes,solution,prime)
neighbor = pd.DataFrame({'d':dist}).d.rolling(2,center=True).sum().values
oldnext = pd.DataFrame({'d':dist}).d.rolling(3,center=True).sum().values
notprime = ~np.isin(solution[:-1], prime)
isten = (np.arange(n)+1)%10==0
isextra = isten&notprime
better = 0

time_a = time.time()

cur = np.append([0],distance(coor[0],coor))
nex = np.append([0],distance(coor[1],coor))

swap = {}
for left in range(1,n-1):
#every left
    pre_coor, left_coor, next_coor = coor[left-1:left+2]
    adj_dist = distance(left_coor, next_coor)
    
    FROM_FLAG = (left+1)%10 == 0
    TO_FLAG = left%10 == 0
    LEFT_FLAG = notprime[left]
    LEFT_pre_FLAG = notprime[left-1]
    LEFT_nex_FLAG = notprime[left+1]
    
    ##old cost
    left_nb = neighbor[left]
    old_cost = neighbor[left:] + left_nb
    old_cost[0] = 0
    old_cost[1] = oldnext[left]

    ##new cost
    pre = cur[1:]
    
    cur = nex[1:]
    
    nex = distance(next_coor,coor[left-1:])

    cur_1 = cur[:-2]
    cur_2 = cur[2:]
    pre_ = pre[1:-1]
    nex_ = nex[1:-1]
    
    if TO_FLAG & LEFT_pre_FLAG:
        pre_ = pre_ + pre_*0.1
    if FROM_FLAG:
        nex_ = nex_ + nex_*notprime[left:]*0.1
        
    cur_1 = cur_1 + cur_1*isextra[left-1:-1]*0.1
    
    if LEFT_FLAG:
        cur_2 = cur_2 + cur_2*isten[left:]*0.1
    
    if FROM_FLAG & LEFT_nex_FLAG:
        adj_dist *= 1.1
    new_cost = pre_ + nex_ + cur_1 + cur_2
    
    
    new_cost[0] = 0
    new_cost[1] += adj_dist
    
    save = old_cost - new_cost
    good = np.where(save>0)[0]
    
    if len(good)>0:
        good_save = save[good]
        swap[left] = [good+left,good_save]
    
    if left%10000 == 0:
        time_b = time.time()
        print(left,time_b-time_a)
        time_a = time_b
print(time.time()-start)
print(len(swap))
###############################################
bk = set()
candidate = []
for left in swap:
    right,improve = swap[left]
    for i in range(len(right)):
        candidate.append((improve[i],right[i],left))
        
candidate = sorted(candidate)[::-1]
l=[]
r=[]
for improve,left,right in candidate:
    fill = [left-1,left,left+1,right-1,right,right+1]
    fill = [not (x in bk) for x in fill]
    if all(fill):
        l.append(left)
        r.append(right)
        bk.add(left-1)
        bk.add(left)
        bk.add(left+1)
        bk.add(right-1)
        bk.add(right)
        bk.add(right+1)

buf = solution[l]
solution[l] = solution[r]
solution[r] = buf

dist = score(nodes,solution,prime).sum()

10000 126.87462496757507
20000 121.92999982833862
30000 117.62800002098083
40000 115.39000010490417
50000 102.39299988746643
60000 98.45900011062622
70000 81.63500022888184
80000 57.76199984550476
90000 51.25800013542175
100000 45.418999671936035
110000 40.664000034332275
120000 37.019999980926514
130000 31.63700032234192
140000 24.470999717712402
150000 19.38700008392334
160000 14.878999948501587
170000 11.11900019645691
180000 7.998999834060669
190000 4.206000089645386
1111.426614522934
4


In [8]:
p.score(p.solution)

1516548.0797786727